In [2]:
# modules
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import numpy as np
import hashlib
from datetime import datetime
import concurrent.futures
import pyarrow.parquet as pq
from sqlalchemy import create_engine

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read in Data
outcomes = dd.read_csv(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\outcomes_df.csv')
street = dd.read_csv(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\street_df.csv')

In [3]:
# Data Exploration
outcomes.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type
0,10370d0a9ab7e145cba38285d0538c52a21f5ffffba798...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.978521,51.346122,On or near OXFORD STREET,E01014801,North Somerset 020C,Suspect charged
1,10370d0a9ab7e145cba38285d0538c52a21f5ffffba798...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.978521,51.346122,On or near OXFORD STREET,E01014801,North Somerset 020C,Suspect charged
2,10370d0a9ab7e145cba38285d0538c52a21f5ffffba798...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.978521,51.346122,On or near OXFORD STREET,E01014801,North Somerset 020C,Suspect charged
3,0ab8300cb6542d05e8b99823a21c2d98c8199620c3a35c...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-3.003506,51.136259,On or near WARREN CLOSE,E01029110,Sedgemoor 009C,Suspect charged
4,2c55cf066bde8fbfa2f21ca6e5463fa7c471cc30847da9...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.530484,51.481025,On or near GUINEA LANE,E01014588,Bristol 013A,Suspect charged


In [4]:
street.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.516919,51.423683,On or near A4175,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
1,NaN,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.509126,51.416137,On or near St Francis Road,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
2,NaN,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.509126,51.416137,On or near St Francis Road,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
3,035b3b13d4c1561d3f7470a512b98d40a9c3287de9ba6e...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.509126,51.416137,On or near St Francis Road,E01014399,Bath and North East Somerset 001A,Other theft,Investigation complete; no suspect identified,NaN
4,c79b016170f909fbb4b1d4fb9be27ad878c0ddf72ceea2...,2014-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.516919,51.423683,On or near A4175,E01014399,Bath and North East Somerset 001A,Violence and sexual offences,Investigation complete; no suspect identified,NaN


In [5]:
# Missing Data
# in outcomes

# no. rows
total_values_outcomes = outcomes.count().compute()

# no. missing values in each col.
missing_values_outcomes = outcomes.isnull().sum().compute()

# % missing in each col.
percentage_missing_outcomes = (missing_values_outcomes / total_values_outcomes) * 100

# Print the statistics separately
print("Total values in Outcomes:")
print(total_values_outcomes)

print("\nMissing values in Outcomes:")
print(missing_values_outcomes)

print("\nPercentage of missing values in Outcomes:")
print(percentage_missing_outcomes)

Total values in Outcomes:
Crime ID        46231550
Month           46231550
Reported by     46231550
Falls within    46231550
Longitude       45261612
Latitude        45261612
Location        46231550
LSOA code       45261433
LSOA name       45261433
Outcome type    46231550
dtype: int64

Missing values in Outcomes:
Crime ID             0
Month                0
Reported by          0
Falls within         0
Longitude       969938
Latitude        969938
Location             0
LSOA code       970117
LSOA name       970117
Outcome type         0
dtype: int64

Percentage of missing values in Outcomes:
Crime ID        0.000000
Month           0.000000
Reported by     0.000000
Falls within    0.000000
Longitude       2.142959
Latitude        2.142959
Location        0.000000
LSOA code       2.143363
LSOA name       2.143363
Outcome type    0.000000
dtype: float64


In [6]:
# Missing Data
# in street

# no. rows
total_values_street = street.count().compute()

# no. missing values in each col.
missing_values_street = street.isnull().sum().compute()

# % missing in each col.
percentage_missing_street = (missing_values_street / total_values_street) * 100

# Print the statistics separately
print("Total values in Street:")
print(total_values_street)

print("\nMissing values in Street:")
print(missing_values_street)

print("\nPercentage of missing values in Street:")
print(percentage_missing_street)

Total values in Street:
Crime ID                 47309040
Month                    63386456
Reported by              63386456
Falls within             63386456
Longitude                62277834
Latitude                 62277834
Location                 63386456
LSOA code                60735981
LSOA name                60735981
Crime type               63386456
Last outcome category    46349326
Context                         0
dtype: int64

Missing values in Street:
Crime ID                 16077416
Month                           0
Reported by                     0
Falls within                    0
Longitude                 1108622
Latitude                  1108622
Location                        0
LSOA code                 2650475
LSOA name                 2650475
Crime type                      0
Last outcome category    17037130
Context                  63386456
dtype: int64

Percentage of missing values in Street:
Crime ID                 33.983814
Month                     0.000

In [7]:
# data types
outcomes.dtypes

Crime ID        string[pyarrow]
Month           string[pyarrow]
Reported by     string[pyarrow]
Falls within    string[pyarrow]
Longitude               float64
Latitude                float64
Location        string[pyarrow]
LSOA code       string[pyarrow]
LSOA name       string[pyarrow]
Outcome type    string[pyarrow]
dtype: object

In [8]:
street.dtypes

Crime ID                 string[pyarrow]
Month                    string[pyarrow]
Reported by              string[pyarrow]
Falls within             string[pyarrow]
Longitude                        float64
Latitude                         float64
Location                 string[pyarrow]
LSOA code                string[pyarrow]
LSOA name                string[pyarrow]
Crime type               string[pyarrow]
Last outcome category    string[pyarrow]
Context                          float64
dtype: object

In [9]:
# test if relationship exists between type of crime and missing values for LSOA name variable.
region_LSOAname_missing = pd.crosstab(street['Crime type'], street['LSOA name'].isna())
print("Relationship between crime type and missingness of LSOA name variable:")
print(region_LSOAname_missing)

Relationship between crime type and missingness of LSOA name variable:
col_0                            False   True 
row_0                                         
Anti-social behaviour         14968191  623615
Bicycle theft                   828072   29737
Burglary                       3474643  117143
Criminal damage and arson      5144303  276224
Drugs                          1560810  114777
Other crime                     887176   57757
Other theft                    4695488  231000
Possession of weapons           392008   20346
Public order                   3795496  103414
Robbery                         657352   23123
Shoplifting                    3325493  141017
Theft from the person           899605   33894
Vehicle crime                  3873067  110890
Violence and sexual offences  16234277  767538


In [10]:
unique_outcomes = outcomes['Outcome type'].unique().compute()

print("Unique values in the 'outcome' column:")
print(unique_outcomes)

Unique values in the 'outcome' column:
0           Action to be taken by another organisation
0                        Defendant sent to Crown Court
0          Formal action is not in the public interest
0                          Unable to prosecute suspect
1              Suspect charged as part of another case
0        Investigation complete; no suspect identified
0                        Offender otherwise dealt with
0                 Offender ordered to pay compensation
1                    Offender given absolute discharge
0                        Offender deprived of property
0             Offender given suspended prison sentence
0                 Offender given conditional discharge
0                                       Offender fined
0                    Offender given community sentence
0                         Court case unable to proceed
0            Offender given a drugs possession warning
0         Further action is not in the public interest
0                         

In [11]:
# Creating Data Frames
outcomes2 = outcomes[['Month', 'Reported by', 'Falls within']]
outcomes3 = outcomes[['Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude']]
street2 = street[['Month', 'Reported by', 'Falls within', 'Crime type']]
street3 = street[['Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Crime type']]

In [12]:
# Drop null from new
outcomes3 = outcomes3.dropna(subset=['Longitude','Latitude'])
street3 = street3.dropna(subset=['Longitude','Latitude'])

In [13]:
# Missing Data in new DFs
# no. rows
total_values_outcomes3 = outcomes3.count().compute()

# no. missing values in each col.
missing_values_outcomes3 = outcomes3.isnull().sum().compute()

# % missing in each col.
percentage_missing_outcomes3 = (missing_values_outcomes3 / total_values_outcomes3) * 100

# Print the statistics separately
print("Total values in Outcomes 3:")
print(total_values_outcomes3)

print("\nMissing values in Outcomes 3:")
print(missing_values_outcomes3)

print("\nPercentage of missing values in Outcomes 3:")
print(percentage_missing_outcomes3)

Total values in Outcomes 3:
Month           45261612
Reported by     45261612
Falls within    45261612
Longitude       45261612
Latitude        45261612
dtype: int64

Missing values in Outcomes 3:
Month           0
Reported by     0
Falls within    0
Longitude       0
Latitude        0
dtype: int64

Percentage of missing values in Outcomes 3:
Month           0.0
Reported by     0.0
Falls within    0.0
Longitude       0.0
Latitude        0.0
dtype: float64


In [14]:
# no. rows
total_values_street3 = street3.count().compute()

# no. missing values in each col.
missing_values_street3 = street3.isnull().sum().compute()

# % missing in each col.
percentage_missing_street3 = (missing_values_street3 / total_values_street3) * 100

# Print the statistics separately
print("Total values in Streets 3:")
print(total_values_street3)

print("\nMissing values in Streets 3:")
print(missing_values_street3)

print("\nPercentage of missing values in Streets 3:")
print(percentage_missing_street3)

Total values in Streets 3:
Month           62277834
Reported by     62277834
Falls within    62277834
Longitude       62277834
Latitude        62277834
Crime type      62277834
dtype: int64

Missing values in Streets 3:
Month           0
Reported by     0
Falls within    0
Longitude       0
Latitude        0
Crime type      0
dtype: int64

Percentage of missing values in Streets 3:
Month           0.0
Reported by     0.0
Falls within    0.0
Longitude       0.0
Latitude        0.0
Crime type      0.0
dtype: float64


In [15]:
# Split "Month" column into separate columns for year and month
outcomes2[['Year', 'Month_Num']] = outcomes2['Month'].str.split('-', n=1, expand=True)
outcomes3[['Year', 'Month_Num']] = outcomes3['Month'].str.split('-', n=1, expand=True)
street2[['Year', 'Month_Num']] = street2['Month'].str.split('-', n=1, expand=True)
street3[['Year', 'Month_Num']] = street3['Month'].str.split('-', n=1, expand=True)

# Convert numeric month to month name
def month_name(x):
    return datetime.strptime(str(x), '%m').strftime('%B')

outcomes2['Month_Name'] = outcomes2['Month_Num'].astype(int).apply(month_name, meta=('Month_Num', 'object'))
outcomes3['Month_Name'] = outcomes3['Month_Num'].astype(int).apply(month_name, meta=('Month_Num', 'object'))
street2['Month_Name'] = street2['Month_Num'].astype(int).apply(month_name, meta=('Month_Num', 'object'))
street3['Month_Name'] = street3['Month_Num'].astype(int).apply(month_name, meta=('Month_Num', 'object'))

# Drop the original "Month" and "Month_Num" columns
outcomes2 = outcomes2.drop(['Month', 'Month_Num'], axis=1)
outcomes3 = outcomes3.drop(['Month', 'Month_Num'], axis=1)
street2 = street2.drop(['Month', 'Month_Num'], axis=1)
street3 = street3.drop(['Month', 'Month_Num'], axis=1)

In [20]:
# save to csv

#outcomes2.to_csv('C:\\Users\\jorda\\Desktop\\portfolio_projects\\outcomes2.csv', index=False)
#outcomes3.to_csv('C:\\Users\\jorda\\Desktop\\portfolio_projects\\outcomes3.csv', index=False)
#street2.to_csv('C:\\Users\\jorda\\Desktop\\portfolio_projects\\street2.csv', index=False)
#street3.to_csv('C:\\Users\\jorda\\Desktop\\portfolio_projects\\street3.csv', index=False)

#print("All DataFrames saved to CSV files.")




In [19]:
# Specify the file paths where you want to save the Parquet files
file_path1 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\outcomes2.parquet'
file_path2 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\outcomes3.parquet'
file_path3 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\street2.parquet'
file_path4 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\street3.parquet'

# Save each DataFrame as a single Parquet file
outcomes2.to_parquet(file_path1, engine='pyarrow', compression='snappy')
outcomes3.to_parquet(file_path2, engine='pyarrow', compression='snappy')
street2.to_parquet(file_path3, engine='pyarrow', compression='snappy')
street3.to_parquet(file_path4, engine='pyarrow', compression='snappy')


In [23]:
partitioned_dir1 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\outcomes2.parquet'

# Read partitioned Parquet files into a single table
table1 = pq.read_table(partitioned_dir1)

# Write the combined table to a single Parquet file
output_file1 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file1.parquet'
pq.write_table(table1, output_file1)


In [25]:
# Specify the directory containing partitioned Parquet files
partitioned_dir2 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\outcomes3.parquet'

# Read partitioned Parquet files into a single table
table2 = pq.read_table(partitioned_dir2)

# Write the combined table to a single Parquet file
output_file2 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file2.parquet'
pq.write_table(table2, output_file2)

In [27]:
# Specify the directory containing partitioned Parquet files
partitioned_dir3 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\street2.parquet'

# Read partitioned Parquet files into a single table
table3 = pq.read_table(partitioned_dir3)

# Write the combined table to a single Parquet file
output_file3 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file3.parquet'
pq.write_table(table3, output_file3)

In [2]:
# Specify the directory containing partitioned Parquet files
partitioned_dir4 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\street3.parquet'

# Read partitioned Parquet files into a single table
table4 = pq.read_table(partitioned_dir4)

# Write the combined table to a single Parquet file
output_file4 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file4.parquet'
pq.write_table(table4, output_file4)

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# Read Parquet files into DataFrames
df1 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file1.parquet')
df2 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file2.parquet')
df3 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file3.parquet')
df4 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file4.parquet')

In [7]:
# Define a function to create table and insert data
def create_table_and_insert_data(df, table_name):
    # Generate SQL CREATE TABLE command based on DataFrame columns
    create_table_sql = f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(MAX)' for col in df.columns])});"
    
    # Execute the CREATE TABLE command
    with engine.connect() as connection:
        connection.execute(create_table_sql)

    # Insert data into the table
    df.to_sql(table_name, engine, if_exists='append', index=False)

In [11]:
# Read the combined Parquet file into a DataFrame
df_combined1 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file1.parquet')

# Specify the output CSV file path
output_csv_file1 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file1.csv'

# Write the DataFrame to a CSV file
df_combined1.to_csv(output_csv_file1, index=False)

print("Data converted to CSV successfully.")


Data converted to CSV successfully.


In [12]:
# Read the combined Parquet file into a DataFrame
df_combined2 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file2.parquet')

# Specify the output CSV file path
output_csv_file2 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file2.csv'

# Write the DataFrame to a CSV file
df_combined2.to_csv(output_csv_file2, index=False)

print("Data converted to CSV successfully.")


Data converted to CSV successfully.


In [3]:
# Read the combined Parquet file into a DataFrame
df_combined3 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file3.parquet')

# Specify the output CSV file path
output_csv_file3 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file3.csv'

# Write the DataFrame to a CSV file
df_combined3.to_csv(output_csv_file3, index=False)

print("Data converted to CSV successfully.")


Data converted to CSV successfully.


In [4]:
# Read the combined Parquet file into a DataFrame
df_combined4 = pd.read_parquet(r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file4.parquet')

# Specify the output CSV file path
output_csv_file4 = r'C:\Users\jorda\Desktop\portfolio_projects\data_sets\uk_crime\combined_file4.csv'

# Write the DataFrame to a CSV file
df_combined4.to_csv(output_csv_file4, index=False)

print("Data converted to CSV successfully.")


Data converted to CSV successfully.
